In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Users\Patri\AppData\Local\Tesseract-OCR\tesseract.exe'
# ^es la forma que encontre de que me ande el tess, perdon por lo bruto :)

#ejemplo de archivo: 'paint1.png'
def import_grayscale_img(archivo):
    img = cv.cvtColor(cv.imread(archivo), cv.COLOR_BGR2GRAY)
    return img

def box_params(h,w,b,ampliar):
    #b es un vector con strings tipo '100', '78'
    #prepara los x,y de las bounding boxes
    box_height = (int(b[4])-int(b[2]))*ampliar
    avg_height = ((h-int(b[4]))+(h-int(b[2])))/2
    box_width = (int(b[3])-int(b[1]))*ampliar
    avg_width = (int(b[3])+int(b[1]))/2
    x1 = int(avg_height - box_height/2)
    x2 = int(avg_height + box_height/2)
    y1 = int(avg_width - box_width/2)
    y2 = int(avg_width + box_width/2)
    if x1<0:
        x1=0
    if x2>h:
        x2=h
    if y1<0:
        y1=0
    if y2>w:
        y2=w
    return x1,x2,y1,y2

def get_boxes(img,ampliar=1,plot=False,grilla=False):
#dada una img, separa los caracteres cada uno en otra image.
#el 'ampliar' es xq a veces, tesseract hace las boxes muy pegadas al caracter.
#Puede servir ponerlo en 1.2, 1.5 aprox
    chars_img = []
    temp = []
    boxes = tess.image_to_boxes(img, lang="eng+equ")
    h, w = img.shape
    for b in boxes.splitlines():
        b = b.split(' ')
        x1,x2,y1,y2 = box_params(h,w,b,ampliar)
        temp = np.copy(img[x1:x2,y1:y2])
        chars_img.append(temp)
    if plot==True:
        plot_img(img)  
        plot_subimg(chars_img,grilla=grilla)
    return chars_img


def plot_img(img):
    plt.imshow(img,'gray')
    plt.show()
    
    
def plot_subimg(subimg_array,grilla=False):
    if grilla==True:
        a = int(np.ceil(np.sqrt(np.shape(subimg_array)[0])))
        i=1
        for image in subimg_array:
            plt.subplot(a,a,i)
            plt.imshow(image,'gray')
            i += 1
        plt.show()
    else:
        for image in subimg_array:
            plt.imshow(image,'gray')
            plt.show()
    

def resize_boxes(h,w,img,write=False,name='prueba1.png'):
    out_img = cv.resize(img,(h,w))
    if write==True:
        cv.imwrite(name,out_img)
    return out_img

In [ ]:
img = import_grayscale_img('paint1.png')
chars_img = get_boxes(img,1.4,plot=True,grilla=True)
out_img = []
i=1
for char_img in chars_img:
    name= 'caracter'+str(i)+'.png'
    i+=1
    temp = resize_boxes(30,50,char_img,write=False,name=name)
    #guarda que si write=True, te guarda por separado todos los caracteres q encuentra
    out_img.append(temp)
plot_subimg(out_img,grilla=True)